In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [3]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Barring my personal details (my name and contact number) shared details can be viewed by andwemet community members.,My religious inclination
0,Male,32,Mumbai,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,
1,Male,36,New Delhi,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,
2,Male,32,Bangalore,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,
3,Female,33,Gurugram,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,
4,Female,32,Mumbai,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Male,44,"Princeton,NJ (USA)/ Mumbai","Supportive family, dependable friends, leaders...",Oldest sibling in a Large and loving family in...,Was married earlier,No,Between 40-50 Lakhs,"Back office role overseas (NJ)bank, now explor...",University of illinois,Masters,Open to making new friends as moving back to I...,I give consent,Hindu
136,Female,50,Bengaluru,Dependable friends,All is well,Was married earlier,Yes,Between 12-15 Lakhs,Entrepreneur,Symbiosis,Masters,Looking for something fruitful,I give consent,Hindu
137,Female,35,Bangalore,All of the above in same order! (having a supp...,It was fast! But it was also in the exact spee...,Never married,No,Between 20-24 Lakhs,Architect,UVCE,Undergraduate,NIL,I give consent,Hindu
138,Male,38,Gurgaon,All of the above - you need Family & Friends t...,They were awesome - have a loving and supporti...,Was married earlier,No,Over 90 Lakhs,Entrepreneur / Fintech,IMT Ghaziabad,PG Diploma,,I give consent,Hindu


In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
0,Male,32,Mumbai,"Separated, filed for a divorce"
1,Male,36,New Delhi,Never married
2,Male,32,Bangalore,Never married
3,Female,33,Gurugram,Was married earlier
4,Female,32,Mumbai,Never married
...,...,...,...,...
135,Male,44,"Princeton,NJ (USA)/ Mumbai",Was married earlier
136,Female,50,Bengaluru,Was married earlier
137,Female,35,Bangalore,Never married
138,Male,38,Gurgaon,Was married earlier


In [ ]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [7]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [8]:
encoded_categorical_features

,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,Widow/ Widower
0,0,1,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,0,0,0,1,0
4,1,0,0,0,0
...,...,...,...,...,...
135,0,0,0,1,0
136,0,0,0,1,0
137,1,0,0,0,0
138,0,0,0,1,0


In [9]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [10]:
dataset

,My Gender,My age,My city of residence,My current relationship status
0,Male,32,Mumbai,"Separated, filed for a divorce"
1,Male,36,New Delhi,Never married
2,Male,32,Bangalore,Never married
3,Female,33,Gurugram,Was married earlier
4,Female,32,Mumbai,Never married
...,...,...,...,...
135,Male,44,"Princeton,NJ (USA)/ Mumbai",Was married earlier
136,Female,50,Bengaluru,Was married earlier
137,Female,35,Bangalore,Never married
138,Male,38,Gurgaon,Was married earlier


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [12]:
dataset

,My Gender,My age,My city of residence,My current relationship status
0,Male,32,Mumbai,"Separated, filed for a divorce"
1,Male,36,New Delhi,Never married
2,Male,32,Bangalore,Never married
3,Female,33,Gurugram,Was married earlier
4,Female,32,Mumbai,Never married
...,...,...,...,...
135,Male,44,"Princeton,NJ (USA)/ Mumbai",Was married earlier
136,Female,50,Bengaluru,Was married earlier
137,Female,35,Bangalore,Never married
138,Male,38,Gurgaon,Was married earlier


In [13]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [15]:
print(features.shape)

(140, 42)


In [16]:
k =100
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=100)

In [17]:
row =9
selectedRow = refDataset.iloc[row]

In [18]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[  9  33  22  89 129   2  69  96  93  17  68  18  15 137  70  57  16   5
   30 115  27 130  14 133  46  10 132  85  40  49  83  75 101 107  60  44
  103  63  97 118   1   7  35  53 119  38  81  91  77  66 121 128 105  73
   50  99 114  48   4 106   6 127 122 117   8  34 131  98  79  94  47  87
   71   0  82  25 125  20  52  28  67  92  64  90 102 116  39 138 109 123
   95 124 134 113  23  61  76  62  29  31]]


In [19]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Female' 35 'Bangalore ' 'Never married']


['Female' 35 'Bangalore' 'Never married']


['Female' 34 'Bangalore' 'Never married']


['Female' 33 'Bangalore' 'Never married']


['Female' 31 'Bangalore' 'Never married']


['Female' 30 'Bangalore' 'Never married']


['Female' 29 'Bangalore/ Chennai ' 'Never married']


['Female' 42 'Bangalore ' 'Never married']


['Female' 32 'Mumbai' 'Never married']


['Female' 32 'Delhi' 'Never married']


['Female' 33 'Delhi' 'Never married']


['Female' 33 'Gurgaon' 'Never married']


['Female' 34 'Gurgaon' 'Never married']


['Female' 34 'Bikaner ' 'Never married']


['Female' 34 'Bangalore' 'Was married earlier']


['Female' 34 'Chicago, IL' 'Never married']


['Female' 29 'Bangalore' 'Was married earlier']


['Female' 35 'Hyderabad ' 'Never married']


['Female' 35 'Gurugram' 'Never married']


['Female' 36 'Bangalore ' 'Was married earlier']


['Female' 36 'Bangalore' 'Was married earlier']


['Female' 36 'Bengaluru' 'Never married']


['Female'

In [22]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [23]:
print(preprocessData(refDataset))

[[ 1.         -1.02096198  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.29034032  1.         ...  0.          0.
   0.        ]
 [ 1.         -1.02096198  1.         ...  0.          0.
   0.        ]
 ...
 [ 0.         -0.65565115  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.65565115  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.65565115  1.         ...  0.          0.
   0.        ]]


In [20]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']